In [1]:
import json
import os
import glob
import matplotlib.pyplot as plt
import shutil

In [2]:

## root on my local computer
# root = 'C:/Users/cloftus/Documents/thermal_baboon/annotations/'

## root on my HiWi computer
root = 'C:/Users/meerkat/Documents/thermal_baboon/DATA/short_temp/'

## root on the server
#root = 'Y:/cloftus/analysis/thermal_baboon_project/'

new_baboon_images_root = root + 'complete_thermal_annotations_2020_10_21/frames_matching_annotations'
new_annotations_root = root + 'complete_thermal_annotations_2020_10_21/annotations/json'
output_folder = root + 'all_coco_annotations'
coco_annotation_file = os.path.join(output_folder, 'baboon_coco_annotations.json')
output_image_folder = os.path.join(output_folder, 'images')
os.makedirs(output_image_folder, exist_ok=True)
baboon_folders = glob.glob(os.path.join(new_baboon_images_root, '*'))[:4]
# print(*baboon_folders, sep='\n')

In [3]:
output_image_folder

'C:/Users/meerkat/Documents/thermal_baboon/DATA/short_temp/all_coco_annotations\\images'

In [4]:
image_files = glob.glob(os.path.join(new_baboon_images_root, '*', '*.tiff'))


In [5]:
image_names = [os.path.basename(f) for f in image_files]
short_names = ['_'.join(name.split('_')[1:]) for name in image_names]




In [6]:
print(len(image_files), len(list(set(image_files))))
print(len(short_names), len(list(set(short_names))))

13 13
13 13


In [7]:
loopy_annotations = glob.glob(new_annotations_root+'/*')[:4]
loopy_annotations

['C:/Users/meerkat/Documents/thermal_baboon/DATA/short_temp/complete_thermal_annotations_2020_10_21/annotations/json\\video_1_20190731_annotation_1326_11171_data.json',
 'C:/Users/meerkat/Documents/thermal_baboon/DATA/short_temp/complete_thermal_annotations_2020_10_21/annotations/json\\video_1_20190731_annotation_1327_18251_data.json']

In [8]:
bad_annotation_count = 0
bad_folders = []
last_bad_image_ids = []

# one id for every image
image_id = 1

file_id_dict = {}
categories = {}
# one id for every category
category_id = 1

# one id for every annotation
annotation_id = 1

annotations_raw = {}

annotations_raw ['info'] = []
annotations_raw ['info'].append({
    'description': 'Raw COCO style baboon annotations',
    'url': '',
    'version': '1.0',
    'year': 2021,
    'contributor': 'Carter Loftus Ben Koger',
    'date_created': '2021/01/26'    
})

annotations_raw ['licenses'] = []
annotations_raw ['licenses'].append({
    'url': '',
    'id': 0,
    'name': '',  
})

annotations_raw['images'] = []

annotations_raw['annotations'] = []

annotations_raw['categories'] = []


for folder_num, folder in enumerate(baboon_folders[:]):
    
    if folder_num % 10 == 0:
        print(f'{folder_num} / {len(baboon_folders)} done. {bad_annotation_count} bad annotations.')
    
    
    loopy_json_file = os.path.join(new_annotations_root, os.path.basename(folder)+'.json')
    
    if os.path.exists( loopy_json_file ):
        image_files = sorted(glob.glob(os.path.join(folder, '*.tiff')))
        for image_file in image_files:
            
            # This was here for a previous fix of a frame that was annotated even though it was just an empty black frame. It isn't needed anymore because I fixed this in the original annotation
            if os.path.basename(image_file) == '038_20190807_161442000000.tiff':
                file_id_dict[int(os.path.basename(image_file).split('_')[0])] = 'skip'
                
                asdfas
                
                continue
                
            im = plt.imread(image_file)
            shutil.copy(image_file, output_image_folder)
            
            file_id_dict[int(os.path.basename(image_file).split('_')[0])] = image_id
            annotations_raw['images'].append({
                'license': 0,
                'file_name': os.path.basename(image_file),
                'coco_url': '',
                'height': im.shape[0],
                'width': im.shape[1],
                'date_captured': '',
                'flickr_url': '',
                'id': image_id
            })
            image_id += 1
        
        with open(loopy_json_file) as json_file:
            loopy_annotations = json.load(json_file)
        
        for annotation in loopy_annotations:
            
            ## this makes all the baboons just tagged as 'baboon' and leaves out their size. Remove this line if we want to run the model considering the baboon's size in the frame
            annotation[ 'class' ] = 'baboon'
            
            if annotation['frame_count'] is None:
                annotation['frame_count'] = annotation['frame_number']
            if annotation['frame_count'] is None:
                
                asdfasdf 
                
                bad_annotation_count += 1
                bad_folders.append(folder)
                last_bad_image_ids.append({'ff': image_id-len(image_files), 'lf': image_id})
                continue
            if file_id_dict[annotation['frame_count']] == 'skip':
                print('skipping frame count {}'.format(annotation['frame_count']))
                continue
            
            segmentation = [annotation['x0'], annotation['y0'],
                            annotation['x1'], annotation['y1'],
                            annotation['x2'], annotation['y2'],
                            annotation['x3'], annotation['y3'],
                           ]
            
            width = annotation['x1'] - annotation['x0']
            height = annotation['y2'] - annotation['y0']
            
            bbox = [annotation['x0'], annotation['y0'],
                    width, height
                    ]
            
            if not annotation['class'] in categories.keys():
                categories[annotation['class']] = category_id
                category_id += 1
            
            annotations_raw['annotations'].append({
                'segmentation': [segmentation],
                'area': width*height,
                'iscrowd': 0,
                'image_id': file_id_dict[int(annotation['frame_count'])],
                'bbox': bbox,
                'category_id': categories[annotation['class']],
                'id': annotation_id
            })  

            annotation_id += 1

for category, category_id in categories.items():
    annotations_raw['categories'].append({
        'supercategory': 'baboon',
        'id': category_id,
        'name': category

    })   
    
print('processed {} annotations in {} images'.format(annotation_id, image_id))



with open(coco_annotation_file, 'w') as outfile:
    json.dump(annotations_raw, outfile, indent=4)

0 / 2 done. 0 bad annotations.
processed 425 annotations in 14 images


In [9]:
# bad_folders_set = []
# last_bad_image_ids

In [10]:
len(last_bad_image_ids)

0

In [11]:
added = []
bad_ids_set = []
for d in last_bad_image_ids:
    if d['ff'] not in added:
        bad_ids_set.append(d)
        added.append(d['ff'])
print(len(bad_ids_set))

0


In [12]:
i = [d['frame_count'] for d in loopy_annotations if d['frame_count'] is None]
len(i)

0

In [13]:
i[:10]

[]

In [14]:
from detectron2.utils.visualizer import Visualizer
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog
import cv2
import random

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 



In [15]:
register_coco_instances("baboons", {}, coco_annotation_file, output_image_folder)

In [16]:
%matplotlib inline

dataset_dicts = DatasetCatalog.get("baboons")
metadata = MetadataCatalog.get("baboons")

counter = 0 

print_dir = root + 'processed_coco_annotations'
os.makedirs( print_dir, exist_ok = True )

os.chdir( print_dir )

for d in dataset_dicts[:]:
    img = cv2.imread(d["file_name"])
#         print(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    plt.figure(figsize=(10,10))
    plt.imshow(vis.get_image())
    plt.title(d["file_name"])
    plt.savefig( f'{counter:05}.png' )
    
    counter += 1
#     plt.figure(figsize=(10,10))
#     plt.imshow(img[:,:,::-1])
    
    
#     cv2.imshow('im', vis.get_image()[:, :, ::-1])
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
# plt.close('all')

In [17]:
with open(coco_annotation_file) as file:
    x = json.load(file)

In [18]:
plt.close('all')

In [19]:
categories

{'baboon': 1}